In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imutils import paths
import cv2
from PIL import Image
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from tensorflow.keras.models import load_model
from joblib import dump, load
import pygame
import time

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
detector= cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [3]:
hat1 = cv2.imread('xmas/hat1.png', -1)
beard1 = cv2.imread('xmas/bea1.png', -1)
hor1 = cv2.imread('xmas/hor1.png', -1)
nose1 = cv2.imread('xmas/nose1.png', -1)
bhat1 = cv2.imread('xmas/bhat2.png', -1)
carr1 = cv2.imread('xmas/carro1.png', -1)
scarf1 = cv2.imread('xmas/scarf1.png', -1)

In [4]:
# Santa filter
def santa_filter(img, x, y, w, h):
    face_width = w
    face_height = h
    
    # hat
    hat2 = cv2.resize(hat1, (int(face_width * 1.2), int(face_height * 1.0)))
    # beard
    beard2 = cv2.resize(beard1, (int(face_width * 1.2), int(face_height * 1.2)))
    
    # hat located
    for i in range(int(face_height * 1.0)):
        for j in range(int(face_width * 1.2)):
            for k in range(3):
                if hat2[i][j][3] != 0:
                    sh = y + i - int(0.7 * h) - 1
                    sw = x + j - int(0.1 * w)
                    if sh < 0 or sw < 0:
                        continue

                    img[sh][sw][k] = hat2[i][j][k]
    
    # beard located
    for i in range(int(face_height * 1.2)):
        for j in range(int(face_width * 1.2)):
            for k in range(3):
                if beard2[i][j][3] != 0:
                    sh = y + i + int(0.55 * h) - 1
                    sw = x + j - int(0.1 * w)
                    if sh < img.shape[0] and sw < img.shape[1]:
                        img[sh][sw][k] = beard2[i][j][k]
    
    return img

In [5]:
# reindeer filter
def reindeer_filter(img, x, y, w, h):
    face_width = w
    face_height = h
    
    # horn
    hor2 = cv2.resize(hor1, (int(face_width * 1.2), int(face_height * 0.6)))
    # nose
    nose2 = cv2.resize(nose1, (int(face_width * 0.5), int(face_height * 0.3)))
    
    # horn located
    for i in range(int(face_height * 0.6)):
        for j in range(int(face_width * 1.2)):
            for k in range(3):
                if hor2[i][j][3] != 0 and hor2[i][j][k] != 0:
                    sh = y + i - int(0.7 * h) - 1
                    sw = x + j - int(0.1 * w)
                    if sh < 0 or sw < 0:
                        continue

                    img[sh][sw][k] = hor2[i][j][k]
    
    # nose located
    for i in range(int(face_height * 0.3)):
        for j in range(int(face_width * 0.5)):
            for k in range(3):
                if nose2[i][j][3] != 0:
                    img[y + i + int(0.45 * h) - 1][x + j + int(0.25 * w)][k] = nose2[i][j][k]
    
    return img

In [6]:
# snowman filter
def snowman_filter(img, x, y, w, h):
    face_width = w
    face_height = h
    
    # hat
    bhat2 = cv2.resize(bhat1, (int(face_width * 1.0), int(face_height * 0.8)))
    # carrot nose
    carr2 = cv2.resize(carr1, (int(face_width * 0.7), int(face_height * 0.3)))
    #scarf
    scarf2 = cv2.resize(scarf1, (int(face_width * 1.8), int(face_height * 1.2)))
    
    # hat located
    for i in range(int(face_height * 0.8)):
        for j in range(int(face_width * 1.0)):
            for k in range(3):
                if bhat2[i][j][3] != 0:
                    sh = y + i - int(0.7 * h) - 1
                    sw = x + j - int(0.1 * w)
                    if sh < 0 or sw < 0:
                        continue

                    img[sh][sw][k] = bhat2[i][j][k]
    
    # carrot nose located
    for i in range(int(face_height * 0.3)):
        for j in range(int(face_width * 0.7)):
            for k in range(3):
                if carr2[i][j][3] != 0:
                    img[y + i + int(0.45 * h) - 1][x + j + int(0.35 * w)][k] = carr2[i][j][k]
    
    # scarf located
    for i in range(int(face_height * 1.2)):
        for j in range(int(face_width * 1.8)):
            for k in range(3):
                if scarf2[i][j][3] != 0:
                    sh = y + i + int(0.8 * h) - 1
                    sw = x + j - int(0.3 * w)
                    if sh < img.shape[0] and sw < img.shape[1]:
                        img[sh][sw][k] = scarf2[i][j][k]       

    return img

In [8]:
# screen shots' indexes
ssid = 1

In [10]:
URL = "http://192.168.178.31:4747/video"
cap = cv2.VideoCapture(0)  # Capture webcam

k = 0

while (True):
    ret, img = cap.read()
    faces = detector.detectMultiScale(img, 1.3, 5)
    
    for (x, y, w, h) in faces:
        if  w  >  130 : 
            try:                             
                if k == 1:
                    img = santa_filter(img, x, y, w, h)

                if k == 2:
                    img = reindeer_filter(img, x, y, w, h)
                
                if k == 3:
                    img = snowman_filter(img, x, y, w, h)


            except Exception as e:
                print("exception", str(e))
    
    key = cv2.waitKey(10)
    
    if key == ord('q'):
        break
    elif key == ord('1'):
        k = 1
    elif key == ord('2'):
        k = 2
    elif key == ord('3'):
        k = 3
    elif key == ord('0'):
        k = 0
    elif key == ord('c'): # press c to capture
        ss = img
        cv2.imwrite('screen shots/ss' + str(ssid) + '.png', ss)
        ssid += 1
        
    cv2.imshow('img', img)

# Kill opencv 
cap.release()
cv2.destroyAllWindows()